In [1]:
import rs_classes.async_request_client as rs
import rs_classes.hooks as hs
import rs_functions.fetch_annotations_meta as fetch_annotations_meta
import rs_functions.fetch_annotation_content as fetch_annotation_content
import rs_functions.mongo_viewer_functions as mvf
import ipywidgets as widgets
from IPython.display import display, HTML
import json

token = ""
base_url = ""

client = rs.AsyncRequestClient(token, base_url)

In [ ]:
annotation_list = [""]

annotations_collection = await fetch_annotations_meta.get_annotation_meta(client, annotation_list)
await fetch_annotation_content.get_annotation_content(client, annotations_collection)
hooks = await mvf.collect_hooks_per_annotation(client, annotations_collection)

In [ ]:
HOOK_TEMPLATE_ID = "39"
BREAK_AFTER_SUCCESSFULL_RESULTS = False
IGNORE_CONDITIONS = True
CHECK_QUEUE_IDS_LIMITATIONS = True
STAGED_PIPELINE = True

TARGET_SCHEMA_ID = ""

for annotation_id, annotation in annotations_collection.items():

    print(f"\033[33m Analysing annotation: {annotation_id}.. \033[0m")      

    for hook in annotation.related_hooks:        

        hook_obj =hooks.get_hook(hook.split("/")[-1])

        if not hook_obj.hook_template:
            continue        

        if hook_obj.hook_template.split("/")[-1] == HOOK_TEMPLATE_ID:             

            configuration_number = 0                                   

            for configuration in hook_obj.settings.get("configurations", None):                                                

                additional_mappings = next((mapping for mapping in configuration.get("additional_mappings", []) if mapping["target_schema_id"] == TARGET_SCHEMA_ID), {})                                
                
                queue_ids = configuration.get("queue_ids", [])
                excluded_queue_ids = configuration.get("excluded_queue_ids", [])

                if CHECK_QUEUE_IDS_LIMITATIONS and queue_ids and (int(annotation.queue) not in queue_ids or int(annotation.queue) in excluded_queue_ids):
                    print("Filtered by Queue ID")
                    continue

                if TARGET_SCHEMA_ID == configuration["mapping"]["target_schema_id"] or TARGET_SCHEMA_ID == additional_mappings.get("target_schema_id"):
                    
                    dataset_key = additional_mappings.get("dataset_key") or configuration["mapping"]["dataset_key"]

                    configuration_number += 1                                        
                    print( f"\033[36m Hook {hook_obj.id} - {hook_obj.name} has configuration #:{configuration_number} with target_schema_id = {TARGET_SCHEMA_ID}, that is mapped to {dataset_key} key in the dataset.\033[0m")                                                                                                    

                    for query in configuration["source"]["queries"]:                                                                        
                        result = {}
                        mvf.find_and_replace_placeholder(query, annotation.annotation_content)                        

                        if query.get("find"):                                                        
                            result = await client.data_storage_find(collectionName=configuration["source"]["dataset"], query=query["find"])
                            query_display = widgets.Textarea(
                                value=json.dumps(query, indent=4),
                                description='Query:',
                                layout=widgets.Layout(width='100%', height='250px'),
                                style={'description_width': 'initial'},
                            )
                            result_display = widgets.Textarea(
                                value=json.dumps(result["result"], indent=4) if result else "No result",
                                description='Result:',
                                layout=widgets.Layout(width='100%', height='250px')
                            )
                            # Arrange the widgets in a horizontal layout
                            hbox_layout = widgets.Layout(display='flex', flex_flow='row', justify_content='space-between', width='100%')
                            hbox = widgets.HBox([query_display, result_display], layout=hbox_layout)

                            display(hbox)                            
                            
                        elif query.get("aggregate"):                                                        
                            if STAGED_PIPELINE and len(query.get("aggregate")) >1:
                                pipeline = []
                                list_pipeline_widgets = []
                                

                                for element in range(1,len(query["aggregate"])):
                                    pipeline.append(query["aggregate"][0:-element])                                
                                
                                for stage in pipeline[::-1]:
                                
                                    pipeline_result = await client.data_storage_aggregate(collectionName=configuration["source"]["dataset"], pipeline=stage)    
                                
                                    
                                    stage_display = widgets.Textarea(
                                        value=json.dumps(stage, indent=4),
                                        description='Query:',
                                        layout=widgets.Layout(width='100%', height='250px'),
                                        style={'description_width': 'initial'},
                                    )
                                    pipeline_result_display = widgets.Textarea(
                                        value=json.dumps(pipeline_result["result"], indent=4) if pipeline_result else "No result",
                                        description='Result:',
                                        layout=widgets.Layout(width='100%', height='250px')
                                    )
                                    list_pipeline_widgets.append(stage_display)
                                    list_pipeline_widgets.append(pipeline_result_display)

                                
                                grid = widgets.GridBox(
                                    children=list_pipeline_widgets,
                                    layout=widgets.Layout(
                                        width='100%',
                                        grid_template_columns='50% 50%',  # Two columns
                                        grid_gap='10px'  # Spacing between cells
                                    )
                                )                                                                   

                                display(grid)

                            else:
                                result = await client.data_storage_aggregate(collectionName=configuration["source"]["dataset"], pipeline=query["aggregate"]) 
                                query_display = widgets.Textarea(
                                    value=json.dumps(query, indent=4),
                                    description='Query:',
                                    layout=widgets.Layout(width='100%', height='250px'),
                                    style={'description_width': 'initial'},
                                )
                                result_display = widgets.Textarea(
                                    value=json.dumps(result["result"], indent=4) if result else "No result",
                                    description='Result:',
                                    layout=widgets.Layout(width='100%', height='250px')
                                )
                                # Arrange the widgets in a horizontal layout
                                hbox_layout = widgets.Layout(display='flex', flex_flow='row', justify_content='space-between', width='100%')
                                hbox = widgets.HBox([query_display, result_display], layout=hbox_layout)

                                display(hbox)                                                       
                                                                                        
                        else:
                            print("No Find or Aggregate has been found")
                        
                                               
                        if result.get("result") and BREAK_AFTER_SUCCESSFULL_RESULTS:
                            print("The result above will be shown in the UI!")
                            break                       